In [1]:
import re
import pandas as pd 
import os 
import jieba
import jieba.analyse

In [15]:
comments_senti_result_path = r"\專案實作\地區餐廳評論分析\google_restaurant_comments_txt\restaurant_comments_processed"

with open(r"\專案實作\地區餐廳評論分析\stop_word2.txt",'r',encoding='utf-8') as file:
    stopwords = list(file.read().splitlines())
    
def remove_stopword(text):
    words = text.split('|')
    filtered_words = [word for word in words if word not in stopwords]
    return '|'.join(filtered_words)

def comments_senti_classified_keywords(data,senti_result,keywords_amount):
    """
    :param data: 要傳入的個別餐廳評論留言情感分析excel 
    :param senti_result: 情感分析結果類別(正面、負面、衝突、中性)
    :param keywords_amount: 設定要回傳的關鍵字詞數量
    :return: 存放關鍵字詞的列表
    """
    data_senti_result = data[data['情感分析結果']==senti_result]
    senti_result_comments_content = ' '.join(data_senti_result['評論留言停用詞去除'])
    senti_result_comments_keywords = jieba.analyse.extract_tags(senti_result_comments_content,topK=keywords_amount)
    return senti_result_comments_keywords

Restaurant_Comments_Senti_Keywords = pd.DataFrame(columns=['RESTAURANT','SENTI_RESULT','KEYWORDS'])

for root, dirs, files in os.walk(comments_senti_result_path):
    # for dir in dirs:
        # print(dir)
    for file_name in files:
        # print(os.path.join(root,file_name))
    
        if file_name.endswith('_result.xlsx'):
            # print(file_name)
            df_senti_result_path = os.path.join(root,file_name)
            df_senti_result = pd.read_excel(df_senti_result_path)
            
            df_senti_result['評論留言停用詞去除'] = df_senti_result['評論留言內容斷詞'].apply(remove_stopword)
            # print(df_senti_result)
            
            positive_keywords_list = comments_senti_classified_keywords(data=df_senti_result,senti_result='正面',keywords_amount=25)
            negative_keywords_list = comments_senti_classified_keywords(data=df_senti_result,senti_result='負面',keywords_amount=25)
            conflict_keywords_list = comments_senti_classified_keywords(data=df_senti_result,senti_result='衝突',keywords_amount=25)
            
            restaurant_name = file_name.replace('Comment_of_','').replace('_result.xlsx','')
            df_senti_keywords = pd.DataFrame(columns=['RESTAURANT','SENTI_RESULT','KEYWORDS'])
            
            df_senti_keywords = df_senti_keywords.append({'RESTAURANT':restaurant_name,'SENTI_RESULT':'正面','KEYWORDS':'|'.join(positive_keywords_list)},ignore_index=True)
            df_senti_keywords = df_senti_keywords.append({'RESTAURANT':restaurant_name,'SENTI_RESULT':'負面','KEYWORDS':'|'.join(negative_keywords_list)},ignore_index=True)
            df_senti_keywords = df_senti_keywords.append({'RESTAURANT':restaurant_name,'SENTI_RESULT':'衝突','KEYWORDS':'|'.join(conflict_keywords_list)},ignore_index=True)
            
            df_senti_keywords.reset_index(drop=True,inplace=True)
            Restaurant_Comments_Senti_Keywords = pd.concat([Restaurant_Comments_Senti_Keywords,df_senti_keywords],ignore_index=True)
            

# 合併給上MEAL_TYPE
google_restaurant_senti_info = pd.read_excel(r"\專案實作\地區餐廳評論分析\data_processed\google_restaurant_senti_info.xlsx")
Restaurant_Comments_Senti_Keywords = pd.merge(Restaurant_Comments_Senti_Keywords,google_restaurant_senti_info[['RESTAURANT','MEAL_TYPE']],on='RESTAURANT',how='left')

Restaurant_Comments_Senti_Keywords.to_excel(r"\專案實作\地區餐廳評論分析\Restaurant_Comments_Senti_Keywords.xlsx",index=False)

# Restaurant_Comments_Senti_Keywords

In [15]:
"""
測試
    停用詞處理
    關鍵字擷取
"""

df = pd.read_excel(r"\專案實作\地區餐廳評論分析\google_restaurant_comments_txt\restaurant_comments_processed\Buffet_restaurant_comments_txt\Comment_of_旭集 和食集錦 信義店_result.xlsx")

# with open(r"\專案實作\地區餐廳評論分析\stop_word2.txt",'r',encoding='utf-8') as file:
#     stopwords = list(file.read().splitlines())
# 
# pattern = '|'.join([re.escape(word) for word in stopwords])
# df['評論留言停用詞去除'] = df['評論留言原內容處理'].apply(lambda x:re.sub(pattern,'',x))

with open(r"\專案實作\地區餐廳評論分析\stop_word2.txt",'r',encoding='utf-8') as file:
    stopwords = list(file.read().splitlines())
    
def remove_stopword(text):
    words = text.split('|')
    filtered_words = [word for word in words if word not in stopwords]
    return '|'.join(filtered_words)

df['評論留言停用詞去除'] = df['評論留言內容斷詞'].apply(remove_stopword)

def comments_senti_classified_keywords(data,senti_result,keywords_amount):
    """
    :param data: 要傳入的個別餐廳評論留言情感分析excel 
    :param senti_result: 情感分析結果類別(正面、負面、衝突、中性)
    :param keywords_amount: 設定要回傳的關鍵字詞數量
    :return: 存放關鍵字詞的列表
    """
    data_senti_result = data[data['情感分析結果']==senti_result]
    senti_result_comments_content = ' '.join(data_senti_result['評論留言停用詞去除'])
    senti_result_comments_keywords = jieba.analyse.extract_tags(senti_result_comments_content,topK=keywords_amount)
    return senti_result_comments_keywords

positive_keywords = comments_senti_classified_keywords(data=df,senti_result='正面',keywords_amount=25)
print(positive_keywords)

negative_keywords = comments_senti_classified_keywords(data=df,senti_result='負面',keywords_amount=25)
print(negative_keywords)

conflict_keywords = comments_senti_classified_keywords(data=df,senti_result='衝突',keywords_amount=25)
print(conflict_keywords)

['餐點', '好吃', '食材', '新鮮', '排隊', '不錯', '旭集', '干貝', '甜點', '飲料', '服務', '生魚片', '動線', '選擇', '海鮮', '候位', '整體', '高級', '訂位', '人員', '日式', '蛤蜊', '食物', '餐廳', '下午茶']
['排隊', '動線', '服務', '時間', '人員', '餐點', '壽司', '服務員', '生魚片', '訂位', '價位', '取餐', '干貝', '態度', '服務生', '14', '菜色', '電話', '餐廳', '不錯', '飲料', '香魚', '壽星', '禮物', '旭集']
['好吃', '動線', '餐點', '不錯', '服務', '甜點', '旭集', '排隊', '食材', '生魚片', '海鮮', '干貝', '新鮮', '時間', '人員', '飲料', '整體', '喜歡', '好喝', '壽司', '土瓶', '餐廳', '取餐', '食物', '味道']


In [14]:
import pandas as pd
import jieba.analyse
import re

# Define the file path
file_path = r"\專案實作\地區餐廳評論分析\google_restaurant_comments_txt\restaurant_comments_processed\Buffet_restaurant_comments_txt\Comment_of_旭集 和食集錦 信義店_result.xlsx"

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Define the path to the stopwords file
stopwords_path = r"\專案實作\地區餐廳評論分析\stop_word2.txt"

# Define a function to remove stopwords from text
def remove_stopword(text):
    words = text.split('|')
    filtered_words = [word for word in words if word not in stopwords]
    return '|'.join(filtered_words)

# Read stopwords into a list
with open(stopwords_path, 'r', encoding='utf-8') as file:
    stopwords = list(file.read().splitlines())

# Apply the remove_stopword function to the '評論留言內容斷詞' column
df['評論留言停用詞去除'] = df['評論留言內容斷詞'].apply(remove_stopword)

# Define a function to extract keywords based on sentiment
def comments_senti_classified_keywords(data, senti_result, keywords_amount):
    data_senti_result = data[data['情感分析結果'] == senti_result]
    senti_result_comments_content = ' '.join(data_senti_result['評論留言停用詞去除'])
    senti_result_comments_keywords = jieba.analyse.extract_tags(senti_result_comments_content, topK=keywords_amount)
    return senti_result_comments_keywords

# Create empty lists to store keywords for each sentiment category
positive_keywords = comments_senti_classified_keywords(data=df, senti_result='正面', keywords_amount=25)
negative_keywords = comments_senti_classified_keywords(data=df, senti_result='負面', keywords_amount=25)
conflict_keywords = comments_senti_classified_keywords(data=df, senti_result='衝突', keywords_amount=25)

# Create a new DataFrame to store the results
result_df = pd.DataFrame(columns=['檔案名稱', '情感分析結果', '關鍵字'])

# Append data for positive sentiment
result_df = result_df.append({'檔案名稱': file_path, '情感分析結果': '正面', '關鍵字': '|'.join(positive_keywords)}, ignore_index=True)

# Append data for negative sentiment
result_df = result_df.append({'檔案名稱': file_path, '情感分析結果': '負面', '關鍵字': '|'.join(negative_keywords)}, ignore_index=True)

# Append data for conflict sentiment
result_df = result_df.append({'檔案名稱': file_path, '情感分析結果': '衝突', '關鍵字': '|'.join(conflict_keywords)}, ignore_index=True)

# Print the resulting DataFrame
result_df

# Export the DataFrame to an Excel file if needed
# result_df.to_excel('result_keywords.xlsx', index=False)

,檔案名稱,情感分析結果,關鍵字
0,C:\Users\NiuNi\OneDrive\桌面\demo\專案實作\地區餐廳評論分析\...,正面,餐點|好吃|食材|新鮮|排隊|不錯|旭集|干貝|甜點|飲料|服務|生魚片|動線|選擇|海鮮|...
1,C:\Users\NiuNi\OneDrive\桌面\demo\專案實作\地區餐廳評論分析\...,負面,排隊|動線|服務|時間|人員|餐點|壽司|服務員|生魚片|訂位|價位|取餐|干貝|態度|服務...
2,C:\Users\NiuNi\OneDrive\桌面\demo\專案實作\地區餐廳評論分析\...,衝突,好吃|動線|餐點|不錯|服務|甜點|旭集|排隊|食材|生魚片|海鮮|干貝|新鮮|時間|人員|...
